# Calculate CER WER metrics OCR 

In [25]:
#!pip3 install evaluate
#!pip3 install jiwer

In [68]:
import os
import sys
import torch
import warnings
from matplotlib import pyplot as plt
from datetime import datetime
from evaluate import load
import pandas as pd
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../../../../')
sys.path.append(NOMEROFF_NET_DIR)

from nomeroff_net import pipeline
from nomeroff_net.tools.ocr_tools import decode_prediction

In [54]:
model_id = "ru"

In [55]:
# auto download latest dataset
from nomeroff_net.tools import modelhub

# auto download latest dataset
info = modelhub.download_dataset_for_model(model_id)
PATH_TO_DATASET = info["dataset_path"]

# local path dataset
#PATH_TO_DATASET = os.path.join(NOMEROFF_NET_DIR, "./data/dataset/TextDetector/ocr_example")

Downloaded model path: /mnt/data/var/www/nomeroff-net/nomeroff_net/tools/../../data/./dataset/TextDetector/ru/autoriaNumberplateOcrRu-2021-09-01.zip


autoriaNumberplateOcrRu-2021-09-01.zip: 1.50GB [02:12, 11.3MB/s]                               


In [57]:
PATH_TO_DATASET

'/mnt/data/var/www/nomeroff-net/nomeroff_net/tools/../../data/./dataset/TextDetector/ru/autoriaNumberplateOcrRu-2021-09-01'

In [58]:
number_plate_detection_and_reading = pipeline("number_plate_detection_and_reading", 
                                              image_loader="opencv")

In [59]:
ocr = number_plate_detection_and_reading.number_plate_text_reading.detector.get_static_module(model_id)

In [60]:
model = ocr.prepare(PATH_TO_DATASET, use_aug=False, num_workers=1)

GET ALPHABET
Max plate length in "val": 9
Max plate length in "train": 9
Max plate length in "test": 9
Letters train  {'K', 'C', 'A', '5', 'X', '6', 'T', 'O', 'B', '2', '8', 'Y', '7', '9', '0', '4', 'E', '1', 'H', 'M', 'P', '3'}
Letters val  {'K', 'C', 'A', '5', 'X', '6', 'T', 'O', 'B', '2', '8', 'Y', '4', '9', '0', '7', 'E', '1', 'H', 'M', 'P', '3'}
Letters test  {'K', 'C', 'A', '5', 'X', '6', 'B', 'O', 'T', '2', '8', 'Y', '4', '7', '0', '9', 'E', '1', 'H', 'M', 'P', '3'}
Max plate length in train, test and val do match
Letters in train, val and test do match
Letters: 0 1 2 3 4 5 6 7 8 9 A B C E H K M O P T X Y
START BUILD DATA


100%|██████████| 2845/2845 [01:19<00:00, 35.73it/s]

DATA PREPARED


In [61]:
ocr.load()

NPOcrNet(
  (conv_nn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True

In [71]:
ocr.test_acc(verbose=True)


[INFO] /mnt/data/var/www/nomeroff-net/nomeroff_net/tools/../../data/./dataset/TextDetector/ru/autoriaNumberplateOcrRu-2021-09-01/test/img/A926AA113.png
Predicted: a926aa13 			 True: a926aa113

[INFO] /mnt/data/var/www/nomeroff-net/nomeroff_net/tools/../../data/./dataset/TextDetector/ru/autoriaNumberplateOcrRu-2021-09-01/test/img/K707KP58_0.png
Predicted: k707kp59 			 True: k707kp58

[INFO] /mnt/data/var/www/nomeroff-net/nomeroff_net/tools/../../data/./dataset/TextDetector/ru/autoriaNumberplateOcrRu-2021-09-01/test/img/K417AE797.png
Predicted: k417ae77 			 True: k417ae797

[INFO] /mnt/data/var/www/nomeroff-net/nomeroff_net/tools/../../data/./dataset/TextDetector/ru/autoriaNumberplateOcrRu-2021-09-01/test/img/E499KP198.png
Predicted: e6499kp198 			 True: e499kp198

[INFO] /mnt/data/var/www/nomeroff-net/nomeroff_net/tools/../../data/./dataset/TextDetector/ru/autoriaNumberplateOcrRu-2021-09-01/test/img/X359KT16.png
Predicted: x359kt16 			 True: x359kt163

[INFO] /mnt/data/var/www/nomeroff

0.9961335676625659

In [65]:
dataset = ocr.dm.test_image_generator
predictions = []
references = []
for idx in tqdm(range(len(dataset)), total=len(dataset), desc="process"):
    img, text = dataset[idx]
    img = img.unsqueeze(0)
    logits = ocr.model(img)
    pred_text = decode_prediction(logits.cpu(), ocr.label_converter)
    references.append(text)
    predictions.append(pred_text)

process:   0%|          | 0/2845 [00:00<?, ?it/s]

In [66]:
cer_score = cer.compute(predictions=predictions, references=references)

In [67]:
wer_score = wer.compute(predictions=predictions, references=references)

In [70]:
pd.DataFrame([{"model": model_id, "CER": cer_score, "WER": wer_score}])

,model,CER,WER
0,ru,0.000502,0.003866
